In [4]:
import pandas as pd
import numpy as np

In [5]:
def get_data_with_xlwings(file_name, sheet):
    data = pd.read_excel(file_name, sheet_name=sheet)
    return data

In [6]:
def load_all_pricing_matrices():
    service_types = [
        "FHD Ground",
        "FHD Ground Canada",
        "FedEx First Overnight",
        "FedEx Priority Overnight",
        "FedEx Standard Overnight",
        "FedEx 2Day A.M.",
        "FedEx 2Day",
        "FedEx Express Saver",
        "Intra-Hawaii Standard List Rate"
    ]
    
    mw_service_types = [
#         ADDRESS STRING FORMATTING AT SOME POINT
        "MW FedEx First Overnight",
        "MW FedEx Priority Overnight",
        "MW FedEx Standard Overnight",
        "MW FedEx 2Day A.M.",
        "MW FedEx 2Day",
        "MW FedEx Express Saver",
        "MWIntra-HawaiiFedExPriorityON"
    ]
    
    freight_service_types = [
        "FedEx First Overnight Freight",
        "FedEx 1Day Freight",
        "FedEx 2Day Freight",
        "FedEx 3Day Freight"
    ]
    
    pricing_matrices = {}
    all_sheets = pd.read_excel("./separated_us_express_rates.xlsx", sheet_name=None)
    
    for service_type in service_types:
        pricing_matrices[service_type] = all_sheets[service_type]
        
    for service_type in mw_service_types:
        pricing_matrices[service_type] = all_sheets[service_type]
        
    for service_type in freight_service_types:
        pricing_matrices[service_type] = all_sheets[service_type]

    return pricing_matrices

In [7]:
def dim_check(l, w, h, dim):
    if np.isnan(l) or np.isnan(w) or np.isnan(h) or np.isnan(dim):
#         IF ANY VALUE IS NOT A NUMBER (nan), CANNOT RUN MATH OPS
        return None
    elif dim == 0:
#         ANY NUM/0 == UNDEFINED
        return None
    else:    
        return l*w*h/dim

In [35]:
def process_data(data):
#     ENSURE HANDLING EXISTS FOR FedEx® Pak
#     MAY NEED TO SEPARATE PRICING FOR ZONES LISTED AS 9-10 AND 11-12 FOR SIMPLICITY OF CALC.

    pricing_matrices = load_all_pricing_matrices()
    
    for index, row in data.iterrows():
        weight = row['RW']
        zone = int(row['Z'])
        service_type = row['Service Type']
        otpub = row['OTPUB']
        fuel_surcharge = row['Fuel Surcharge']
        svc_pkging = row['Service Packaging']
        l = row['L']
        w = row['W']
        h = row['H']
        dim = row['DIM']
        this_dim = dim_check(l, w, h, dim)
        
#         FEDEX STANDARD OVERNIGHT AND FEDEX 2DAY AM HAVE NO DATA FOR ZONES 10-12
#         FEDEX EXPRESS SAVER HAS NO DATA IN ZONE COLUMNS 9-10/11-12/13-16
        if svc_pkging == "FedEx Pak":
            print("NO RATES PROVIDED TO CALCULATE PRICE")
#             otpub.color = (255, 0, 0)
        elif service_type in pricing_matrices:
#             GRAB CORRESPONDING DATA
            pricing_matrix_data = pricing_matrices[service_type]
#             DETERMINE THE LARGER WEIGHT FOR CALC
            actual_weight = weight if this_dim == None or weight > this_dim else this_dim
#             IF ITS AN ENVELOPE
            if actual_weight == 0.5:
                new_otpub = pricing_matrix_data.iloc[0, zone-1] 
                otpub = float("{:.2f}".format(new_otpub))
                print(otpub)
            else:
                new_otpub = pricing_matrix_data.iloc[int(actual_weight)+1, zone-1] 
                otpub = float("{:.2f}".format(new_otpub))
                print(otpub)

        elif service_type == "Ground":
            if row["Recipient Country/Territory"] != "US":
                pricing_matrix_data = pricing_matrices["FHD Ground Canada"]
                actual_weight = weight if this_dim == None or weight > this_dim else this_dim
                zone_index = int()
                
                if zone == 51:
                    zone_index = 1
                elif zone == 54:
                    zone_index = 2
                new_otpub = pricing_matrix_data.iloc[int(actual_weight)-1, zone_index]
                otpub = float("{:.2f}".format(new_otpub))
            else:
                pricing_matrix_data = pricing_matrices["FHD Ground"]
                actual_weight = weight if this_dim == None or weight > this_dim else this_dim
                zone_index = zone
                
                if zone == 14:
                    zone_index = 10
                elif zone == 17:
                    zone_index = 11
                elif zone == 22:
                    zone_index = 12
                elif zone == 23:
                    zone_index = 13
                elif zone == 25:
                    zone_index = 14
                elif zone == 92:
                    zone_index = 15
                elif zone == 96:
                    zone_index = 16
                    
                new_otpub = pricing_matrix_data.iloc[int(actual_weight)-1, zone_index-1]
                otpub = float("{:.2f}".format(new_otpub))

        else:
#             BELOW PRINTS ANY SERVICE TYPE FOR WHICH WE MAY HAVE NOT HAVE ANY HANDLING 
            print(f'{service_type} not addressed!')
#             BELOW CHANGES CELL TO COLOR RED TO EASILY LOCATE UNADRESSED CELLS
            otpub.color = (255, 0, 0)
    
#     AT THIS POINT, THE DATA MANIPULATED IS SIMPLY AN INSTANCE AND LATER WE MUST USE save_results_with_xlwings() FOR
#     MANIPULATED DATA TO PERSIST IN A WORKBOOK
    return data

In [36]:
def save_results_with_xlwings(data, file_name, sheet_name):
    print(data)
#     wb = xw.Book(file_name)
#     ws = wb.sheets[sheet_name]
#     ws.range('A1').options(index=False).value = data
#     wb.save()
#     wb.close()

In [37]:
#BEGIN MAIN SCRIPT USING ABOVE FUNCTIONS
file_name = './charges.xlsm'
spreadsheet_name = 'FREIGHT & ACCESSORIALS'
charge_data = get_data_with_xlwings(file_name, spreadsheet_name)
processed_data = process_data(charge_data)
# save_results_with_xlwings(processed_data, file_name, spreadsheet_name)

NO RATES PROVIDED TO CALCULATE PRICE
89.47
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
48.18
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
32.63
57.43
56.66
48.18
32.63
55.7
NO RATES PROVIDED TO CALCULATE PRICE
69.05
NO RATES PROVIDED TO CALCULATE PRICE
67.11
55.7
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
48.18
NO RATES PROVIDED TO CALCULATE PRICE
80.37
32.63
56.66
39.36
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO CALCULATE PRICE
78.4
NO RATES PROVIDED TO CALCULATE PRICE
32.63
NO RATES PROVIDED TO CALCULATE PRICE
39.36
39.36
39.36
40.49
39.36
39.36
54.16
32.63
91.32
NO RATES PROVIDED TO CALCULATE PRICE
40.49
NO RATES PROVIDED TO CALCULATE PRICE
NO RATES PROVIDED TO 